In [3]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch

Populating the interactive namespace from numpy and matplotlib


In [12]:
overrides = '--num_training_episodes 5_000_000 --eval_every 500_000 --iterate_br true --br_overrides "--num_training_episodes 2_000_000"' 
CONFIG_DIR = '/apps/open_spiel/open_spiel/python/examples/notebooks/configs/old_jan13'
dispatch.dispatch_experiments(CONFIG_DIR, overrides=overrides, game_name='small_game_2', base_job_name='jan25_small_gpu')
# dispatch.dispatch_experiments(CONFIG_DIR, overrides=overrides, game_name='medium_game_2', base_job_name='jan14_medium')
# dispatch.dispatch_experiments(CONFIG_DIR, overrides=overrides, game_name='big_game_2', base_job_name='jan14_big')

Dispatched 6 experiments!


In [13]:
overrides = '--num_training_episodes 5_000_000 --eval_every 500_000 --eval_every_early 50_000 --iterate_br true --br_overrides "--num_training_episodes 1_000_000"' 
CONFIG_DIR = '/apps/open_spiel/open_spiel/python/examples/notebooks/configs/feb2'
# dispatch.dispatch_experiments(CONFIG_DIR, overrides=overrides, game_name='small_game_2', base_job_name='jan25_small_gpu')
# dispatch.dispatch_experiments(CONFIG_DIR, overrides=overrides, game_name='medium_game_2', base_job_name='jan25_medium')
dispatch.dispatch_experiments(CONFIG_DIR, overrides=overrides, game_name='large_game_2', base_job_name='feb2_large')

Dispatched 4 experiments!


In [65]:
overrides = '--num_training_episodes 5_000_000 --eval_every 500_000 --eval_every_early 50_000 --eval_zero true --br_portfolio /apps/open_spiel/open_spiel/python/examples/notebooks/configs/feb4 --iterate_br true --br_overrides "--num_training_episodes 1_000_000"' 
CONFIG_DIR = '/apps/open_spiel/open_spiel/python/examples/notebooks/configs/feb2'
dispatch.dispatch_experiments(CONFIG_DIR, overrides=overrides, game_name='large_game_2', base_job_name='feb5_large')

Dispatched 2 experiments!


In [88]:
# !squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "_mlp  R" | awk '{print $1}' | xargs scancel

In [90]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R"

             JOBID                                                              NAME ST       TIME  NODES NODELIST(REASON)


In [ ]:
# !scancel 5306 5307

In [63]:
!scancel -u ubuntu

In [60]:
# Tiny example for pipeline verification

overrides = '--num_training_episodes 5000 --eval_every 1000 --iterate_br true --br_portfolio /apps/open_spiel/open_spiel/python/examples/notebooks/configs/feb4 --br_overrides "--num_training_episodes 1000"' 
CONFIG_DIR = '/apps/open_spiel/open_spiel/python/examples/notebooks/configs/feb2'
dispatch.dispatch_experiments(CONFIG_DIR, overrides=overrides, game_name='tiny', base_job_name='jan13_tiny_2')


Dispatched 2 experiments!


In [17]:
import pickle
pickle.load(open('/shared/outputs/feb2_large/lstm_deep/evaluations/rewards_checkpoint_3500000.pkl', 'rb'))['walltime']

1183.4480459690094